#Predict NBA Playoff

In [2]:
import pandas as pd

In [3]:
# Load the data
nba_data = pd.read_csv('/content/nba_team_data.csv')

In [5]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing in

In [6]:
import gradio as gr
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [7]:
# Preprocess the data for modeling
nba_data['Playoffs'] = nba_data['Playoffs'].map({'Y': 1, 'N': 0})
nba_data['W/L%'] = nba_data['W/L%'].astype(float)
X = nba_data[['W', 'L', 'W/L%', 'GB', 'PS/G', 'PA/G', 'SRS']]
y = nba_data['Playoffs']

In [8]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [10]:
# Inspect the first few rows of the data
print(X_train.head())

# Check for non-numeric values
for col in X_train.columns:
    print(f"Column '{col}' unique values:", X_train[col].unique())


     W   L   W/L%    GB   PS/G   PA/G   SRS
40  29  53  0.354  30.0  104.5  108.0 -3.53
4   43  39  0.524  10.0  103.2  104.0 -1.23
43  25  57  0.305  34.0  103.4  108.2 -4.92
19  51  31  0.622  16.0  100.7   96.8  4.00
34  48  34  0.585  11.0  105.6  104.2  1.18
Column 'W' unique values: [29 43 25 51 48 22 33 27 67 31 37 59 32 61 34 35 52 46 28 24 49 36 44 40
 41 21 47 20 26 39]
Column 'L' unique values: [53 39 57 31 34 60 49 55 15 51 45 23 50 21 48 47 30 36 54 58 33 46 38 42
 41 61 35 62 56 43]
Column 'W/L%' unique values: [0.354 0.524 0.305 0.622 0.585 0.268 0.402 0.329 0.817 0.378 0.451 0.72
 0.39  0.744 0.415 0.427 0.634 0.561 0.341 0.293 0.598 0.439 0.537 0.488
 0.5   0.256 0.573 0.244 0.317 0.476]
Column 'GB' unique values: ['30.0' '10.0' '34.0' '16.0' '11.0' '43.0' '38.0' '—' '36.0' '35.0' '6.0'
 '33.0' '22.0' '7.0' '19.0' '31.0' '2.0' '24.0' '23.0' '15.0' '27.0'
 '17.0' '26.0' '44.0' '20.0' '12.0' '32.0' '41.0' '18.0']
Column 'PS/G' unique values: [104.5 103.2 103.4 100.7 105.

In [11]:
# Replace non-numeric values with NaN, then fill NaN with appropriate values or drop rows
X_train.replace('—', pd.NA, inplace=True)
X_test.replace('—', pd.NA, inplace=True)

# Convert columns to numeric, forcing errors to NaN
X_train = X_train.apply(pd.to_numeric, errors='coerce')
X_test = X_test.apply(pd.to_numeric, errors='coerce')

# Optionally fill NaN values with a specific value (e.g., 0) or drop rows with NaN values
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)


In [12]:
from sklearn.preprocessing import StandardScaler

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [13]:
# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [14]:
# Train a logistic regression model
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

LogisticRegression()

In [15]:
# Evaluate the model
y_pred = model.predict(X_test_scaled)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.9444444444444444


In [16]:
# Define a function for Gradio interface
def predict_playoff(W, L, WL_percent, GB, PS_G, PA_G, SRS):
    features = pd.DataFrame([[W, L, WL_percent, GB, PS_G, PA_G, SRS]], columns=X.columns)
    features_scaled = scaler.transform(features)
    prediction = model.predict(features_scaled)[0]
    return "Yes" if prediction == 1 else "No"

In [18]:
# Create Gradio interface
iface = gr.Interface(
    fn=predict_playoff,
    inputs=[
        gr.Number(label="Wins"),
        gr.Number(label="Losses"),
        gr.Number(label="Win/Loss Percentage"),
        gr.Number(label="Games Behind"),
        gr.Number(label="Points Scored per Game"),
        gr.Number(label="Points Allowed per Game"),
        gr.Number(label="SRS")
    ],
    outputs="text",
    title="NBA Playoff Predictor",
    description="Predict if a team will make the NBA playoffs based on performance metrics."
)

In [19]:
# Launch the Gradio app
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://96cd8804a932055d8c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
